In [ ]:
using Optics_in_the_length_gauge

In [ ]:
cp = Computation_presets(        # comp presets
    xbounds = [0, 2π/√3],        # adimensional, 
    ybounds = [-2π, 2π],         # adimensional, 
    ωlist = collect(-10:0.1:10), # in eV, 
    broadening = 0.025,          # in eV
    evals = 1e6                  # number of iterations
    )

Quick note: 

Structs in Optics_in_the_lenght_gauge uses Parameters,
so they can be dynamically created using the following syntax.
cp2 = Computation_presets(cp, evals = 1e5), which keeps the rest
of fields as in cp. 

### Bands MLG

<img src="MLG_bands.jpg" alt="Alt text" width="400"/>

### DOS

In [ ]:

μ = 0
h(q) = Presets.MLG_hamiltonian(μ, q) # system specific

# DOS_presets
dos_presets = DOS_presets(
    h = h, 
    computation = cp            #
)

In [ ]:
# Calculation
ω, j = dos(dos_presets)

In [ ]:
fig = Figure(); ax = Axis(fig[1,1], xlabel = "ω (eV)", ylabel = "dos (a.u.)")
lines!(ax, ω, j)

<img src="MLG_dos.jpg" alt="Alt text" width="400"/>

Note that the broadening, $\eta\neq0$ introduces a finite spectral weight at $\omega = 0$ and tails at the band edges

### JDOS

In [ ]:
# Calculation, at two Fermi levels
h0(q) = Presets.MLG_hamiltonian(μ, q)
ω, j0 = jdos(JDOS_presets(h0, cp))

h1(q) = Presets.MLG_hamiltonian(μ + 1, q)
ω, j0 = jdos(JDOS_presets(h1, cp))

In [ ]:
fig = Figure(); ax = Axis(fig[1,1], xlabel = "ω (eV)", ylabel = "jdos (a.u.)")
lines!(ax, ω, j0, label = "μ = 0")
lines!(ax, ω, j1, label = "μ = 1 eV")
axislegend()
fig

<img src="MLG_jdos.jpg" alt="Alt text" width="400"/>

## Linear optical conductivity

In [ ]:
# Presets
dirJ = :x
dirE = :x
h(q) = Presets.MLG_hamiltonian(μ, q)
nabla_h(q) = Presets.MLG_nabla(q) # vector containing the partial derivatives of the Hamiltonian along the periodic directions
sigma_ij_presets = σij_presets(dirJ, dirE, h, nabla_h, cp)

In [ ]:

linear_optical_conductivity(sigma_ij_presets)

In [ ]:
# Calculation of the quantized conductivity of a single Dirac cone
# In this case we specify manually the boundaries of the integration area
# to describe the low energy <1eV physics around one K point

n = 5
ϵ2 = 2π/(3√3)/n
xboundsDP = K1[1] .+ [-ϵ2, ϵ2]
yboundsDP = K1[2] .+ [-ϵ2, ϵ2]
broadening = 0.01
evals = 500000
ωlist = collect(0:.025:0.5)

ω, j_single_DP = linear_optical_conductivity(dirJ, dirE, h, dh,
    xboundsDP, yboundsDP, ωlist,
    η = broadening, evals = evals)

ω, jDP2 = linear_optical_conductivity(dirJ, dirE, h, dh,
    xboundsDP, yboundsDP, ωlist,
    η = broadening/2, evals = evals)

In [ ]:
# Plot
# the 16 prefactor is to express the conductivity
# in as e^2/(16ħ): σxx value of a single Dirac cone

fig = Figure(); ax = Axis(fig[1,1], xlabel = "ω (eV)", ylabel = "σ_xx [e^2/16ħ]")
lines!(ax, ω, 16 .* jDP2, label = "η = 5 meV") 
lines!(ax, ω, 16 .* jDP, label = "η = 10 meV")
axislegend(position = :rb)

<img src="quantized_conductivity_singleCone.jpg" alt="Alt text" width="400"/>

### Tests

In [ ]:
# Testing the bounds

# Full BZ
xboundsFBZ =  2π/√3 .* [0, 1]
ybounds =  [-2π, 2π]


# doubled  BZ
xboundsdoubled =  2π/√3 .* [0, 2]
ybounds =  [-2π, 2π]

# doubled  shift
ϵ = 0.1
xboundsdoubledshift =  2π/√3 .* ([0, 2] .+ ϵ)
ybounds =  [-2π, 2π]

# BZ displaced in xarr
xboundsdisFBZ =  2π/√3 .* [0.5, 1.5]
ybounds =  [-2π, 2π]

# Expanded BZ
xboundsEXP =  4 * 2π/√3 .* [0, 1]
yboundsEXP =  4 .* [-2π, 2π]

# Focus around a DC
n = 1
ϵ2 = 2π/(3√3)/n
xboundsDP = K1[1] .+ [-ϵ2, ϵ2]
yboundsDP = K2[2] .+ [-ϵ2, ϵ2]


ω, jDP = dos(h, xboundsDP, yboundsDP, collect(-10:.1:10),
     η = 0.025, evals = 500000)

ω, jFBZ = dos(h, xboundsFBZ, ybounds, collect(-10:.1:10),
     η = 0.025, evals = 500000)

ω, jdisFBZ = dos(h, xboundsdisFBZ, ybounds, collect(-10:.1:10),
     η = 0.025, evals = 500000)

ω, jdoubled = dos(h, xboundsdoubled, ybounds, collect(-10:.1:10),
     η = 0.025, evals = 500000)

ω, jdoubledshift = dos(h, xboundsdoubledshift, ybounds, collect(-10:.1:10),
     η = 0.025, evals = 500000)

ω, jdisEXP = dos(h, xboundsEXP, yboundsEXP, collect(-10:.1:10),
     η = 0.025, evals = 500000)


![Alt text](comparison.png)

![Alt text](Diracpointsurroundings.png)

In [ ]:
# Tests linear optical conductivity as a function of the integration antisymmetrized_realsum
# Focus around a DC
ϵ2 = 2π/(3√3)/n
xboundsDP = K1[1] .+ [-ϵ2, ϵ2]
yboundsDP = K2[2] .+ [-ϵ2, ϵ2]

ω, jDP = linear_optical_conductivity(dirJ, dirE, h, dh,
    xboundsDP, yboundsDP, collect(0:.1:1),
    η = 0.025, evals = 500000)

![Alt text](comparison2.png)

# New API

In [ ]:
# Comp presets
cp = Computation_presets(
    xbounds = [0, 2π/√3],        # adimensional, 
    ybounds = [-2π, 2π],         # adimensional, 
    ωlist = collect(-10:0.1:10), # in eV, 
    broadening = 0.025,          # in eV
    evals = 1e6                  # number of iterations
    )

### DOS

In [ ]:
# DOS_presets
μ = 0
h(q) = MLG_hamiltonian(μ, q)
dos_presets = DOS_presets(h, cp)

In [ ]:
# Calculation
ω, j = dos(dos_presets)